In [1]:
from time import time, localtime
import numpy as np
from utils import plot_graph
import tensorflow_federated as tff
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras import losses, metrics, optimizers
import random
import pandas as pd
import sys
import matplotlib.pyplot as plt
import nest_asyncio
from pathlib import Path
from checkpoint_manager import FileCheckpointManager
import tensorflow_addons as tfa
nest_asyncio.apply()
SEED = 1337
tf.random.set_seed(SEED)



2022-10-31 01:27:50.555974: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-31 01:27:50.707872: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-31 01:27:50.707894: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-31 01:27:50.736975: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-31 01:27:51.434170: W tensorflow/stream_executor/pla

In [2]:
from models.CNN import create_keras_model


In [3]:
experiment_name = "GQUIC"
method = "CNN"
client_lr = 1
server_lr = 1
NUM_ROUNDS = 1
BATCH_SIZE = 1
split = 5


In [4]:
this_dir = Path.cwd()
model_dir = this_dir / "sdn_saved_models" / experiment_name / method
output_dir = this_dir / "sdn_results" / experiment_name / method

if not model_dir.exists():
    model_dir.mkdir(parents=True)

if not output_dir.exists():
    output_dir.mkdir(parents=True)


In [5]:
def sec_to_hours(seconds):
    a = seconds//3600
    b = (seconds % 3600)//60
    c = (seconds % 3600) % 60
    d = "{:.0f} hours {:.0f} mins {:.0f} seconds".format(a, b, c)
    return d


## Data

In [6]:
data = pd.read_csv(
    '/home/onos/FL/Data Processing/GQUIC_data.csv', index_col=False)


In [7]:
NUM_FEATURE = len(data.columns)-1
NUM_CLASSES = len(np.unique(data['Label']))
total_data_count = data.shape[0]
data_per_set = int(np.floor(total_data_count/split))


In [8]:
test = pd.read_csv(
    '/home/onos/FL/Data Processing/GQUIC_data_test.csv', index_col=False)

In [9]:
x_test = test.drop('Label', axis=1).to_numpy()
y_test = test.Label.to_numpy()

In [10]:
data = data.sample(frac=1).reset_index(drop=True)


In [11]:
DataFrameDict = {}
for i in range(1, split+1):
    client_name = "client_" + str(i)
    start = data_per_set * (i-1)
    end = data_per_set * i

    print(f"Adding data from {start} to {end} for client : {client_name}")
    DataFrameDict[client_name] = data[start:end]


Adding data from 0 to 188844 for client : client_1
Adding data from 188844 to 377688 for client : client_2
Adding data from 377688 to 566532 for client : client_3
Adding data from 566532 to 755376 for client : client_4
Adding data from 755376 to 944220 for client : client_5


In [12]:
# def sklearn_to_df(sklearn_dataset):
#     df = pd.DataFrame(sklearn_dataset.data,
#                       columns=sklearn_dataset.feature_names)
#     df['Label'] = pd.Series(sklearn_dataset.Label)
#     return df


In [13]:
# from sklearn import datasets
# df = sklearn_to_df(datasets.load_iris())


### Exploratory Analysis

In [14]:
print(data['Label'].value_counts())


3    463260
2    261747
1    169592
0     49622
Name: Label, dtype: int64


In [15]:
for df in DataFrameDict.keys():
    print(df)
    print(DataFrameDict[df]['Label'].value_counts())


client_1
3    92876
2    52229
1    33703
0    10036
Name: Label, dtype: int64
client_2
3    92495
2    52564
1    33980
0     9805
Name: Label, dtype: int64
client_3
3    92618
2    52286
1    33954
0     9986
Name: Label, dtype: int64
client_4
3    92285
2    52453
1    34171
0     9935
Name: Label, dtype: int64
client_5
3    92985
2    52215
1    33784
0     9860
Name: Label, dtype: int64


## Federated Learning Approach

### Data Loading

In [16]:
def make_tf_dataset(dataframe, batch_size=None):

    y = dataframe.pop('Label')

    # Dataset creation
    dataset = tf.data.Dataset.from_tensor_slices(
        (dataframe.values, y.to_frame().values))
    dataset = dataset.shuffle(2048, seed=SEED)
    if batch_size:
        dataset = dataset.batch(batch_size)

    return dataset


In [17]:
train_data, val_data = [], []
for client_data in DataFrameDict.keys():
    train_df, val_df = train_test_split(
        DataFrameDict[client_data], test_size=0.2, random_state=SEED)

    # Scaling (Standardization actually hurts performance)
    scaler = MinMaxScaler()
    train_features = scaler.fit_transform(train_df.drop(['Label'], axis=1))
    val_features = scaler.transform(val_df.drop(['Label'], axis=1))

    train_df[train_df.columns.difference(['Label'])] = train_features
    val_df[val_df.columns.difference(['Label'])] = val_features

    # TF Datasets
    train_data.append(make_tf_dataset(train_df, batch_size=BATCH_SIZE))
    val_data.append(make_tf_dataset(val_df, batch_size=1))


2022-10-31 01:28:18.975111: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-31 01:28:18.975152: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-31 01:28:18.975168: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (onos-virtual-machine): /proc/driver/nvidia/version does not exist
2022-10-31 01:28:18.975516: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Model Definition

In [18]:
def input_spec():
    return (
        tf.TensorSpec([None, NUM_FEATURE], tf.float64),
        tf.TensorSpec([None, 1], tf.int64)
    )


In [19]:
def model_fn():
    model = create_keras_model(NUM_CLASSES)

    return tff.learning.from_keras_model(
        model,
        input_spec=input_spec(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


### Training

Each time the `next` method is called, the server model is broadcast to each client using a broadcast function. For each client, one epoch of local training is performed. Each client computes the difference between the client model after training and the initial broadcast model. These model deltas are then aggregated at the server using some aggregation function.

In [20]:
tff_train_acc = []
tff_train_loss = []
tff_val_acc = []
tff_val_loss = []


In [21]:
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(
        learning_rate=client_lr),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(
        learning_rate=server_lr)
)


ValueError: in user code:

    File "/home/onos/FL/venv/lib/python3.9/site-packages/tensorflow_federated/python/learning/templates/model_delta_client_work.py", line 175, in reduce_fn  *
        output = model.forward_pass(batch, training=True)
    File "/home/onos/FL/venv/lib/python3.9/site-packages/tensorflow_federated/python/learning/framework/dataset_reduce.py", line 33, in _dataset_reduce_fn  *
        return dataset.reduce(initial_state=initial_state_fn(), reduce_func=reduce_fn)
    File "/home/onos/FL/venv/lib/python3.9/site-packages/tensorflow_federated/python/learning/keras_utils.py", line 458, in forward_pass  *
        return self._forward_pass(batch_input, training=training)
    File "/home/onos/FL/venv/lib/python3.9/site-packages/tensorflow_federated/python/learning/keras_utils.py", line 411, in _forward_pass  *
        predictions = self.predict_on_batch(inputs, training)
    File "/home/onos/FL/venv/lib/python3.9/site-packages/tensorflow_federated/python/learning/keras_ut

In [ ]:
evaluator = tff.learning.build_federated_evaluation(model_fn)


In [ ]:
state = iterative_process.initialize()


In [ ]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%y_%m_%d_%H")
print("Current Time =", current_time)


In [ ]:
ckpt_manager = FileCheckpointManager(
    model_dir/current_time/"{}_{}".format(experiment_name,method))


In [ ]:
start = time()
for i in range(NUM_ROUNDS):
    # Train
    result = iterative_process.next(state, train_data)
    state = result.state
    train_metrics = result.metrics['client_work']['train']

    # Validation
    federated_metrics = evaluator(result.state.global_model_weights, val_data)
    val_metrics = federated_metrics['eval']

    # Metrics
    train_loss = train_metrics['loss']
    train_acc = train_metrics['sparse_categorical_accuracy']
    val_loss = val_metrics['loss']
    val_acc = val_metrics['sparse_categorical_accuracy']

    # Print
    print('round {:2d}\ntrain_loss={l:.3f}, train_acc={ac:.3f}'.format(
        i+1, l=train_loss, ac=train_metrics['sparse_categorical_accuracy']))
    print('val_loss: {:.3f} val_acc: {:.3f}'.format(
        val_loss, val_acc))

    # Save
    ckpt_manager.save_checkpoint(state, round_num=i)
    # logs
    tff_train_acc.append(float(train_metrics['sparse_categorical_accuracy']))
    tff_train_loss.append(float(train_metrics['loss']))
    tff_val_acc.append(float(val_metrics['sparse_categorical_accuracy']))
    tff_val_loss.append(float(val_metrics['loss']))

end = time() - start


In [ ]:
federated_metrics

In [ ]:
total_time = "Time: {}".format(sec_to_hours(end))
print(total_time)


In [ ]:
train_val = str(round(train_acc*100)) + "_" + str(round(val_acc*100))


In [ ]:
this_dir = Path.cwd()
model_dir = this_dir / "sdn_saved_models" / experiment_name / method / train_val
output_dir = this_dir / "sdn_results" / experiment_name / method / train_val

if not model_dir.exists():
    model_dir.mkdir(parents=True)

if not output_dir.exists():
    output_dir.mkdir(parents=True)


### Evaluation

In [ ]:
with open(output_dir/'parameters.txt', 'w') as f:
    print('client_lr: {}\nserver_lr: {}\nRounds: {}\nBATCH_SIZE: {}'.format(
        client_lr, server_lr, NUM_ROUNDS, BATCH_SIZE), file=f)
    f.close()


In [ ]:
text_file = open(output_dir/"time.txt", "w")
n = text_file.write(total_time)
text_file.close()


In [ ]:
model_for_inference = create_keras_model(NUM_CLASSES)
predictions = model_for_inference.predict_on_batch(x_test)
y_pred = np.argmax(predictions, axis=-1)
classes = []
for c in range(NUM_CLASSES):
    classes.append("Class {}".format(c))
print(classes)
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
with open(output_dir/'metrics.txt', 'w') as f:
    confusion = confusion_matrix(y_test, y_pred)
    print('Confusion Matrix\n', file=f)
    print(confusion, file = f)

    # importing accuracy_score, precision_score, recall_score, f1_score
    print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)), file=f)

    print('Micro Precision: {:.2f}'.format(
        precision_score(y_test, y_pred, average='micro')), file=f)
    print('Micro Recall: {:.2f}'.format(
        recall_score(y_test, y_pred, average='micro')), file=f)
    print(
        'Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')), file=f)

    print('Macro Precision: {:.2f}'.format(
        precision_score(y_test, y_pred, average='macro')), file=f)
    print('Macro Recall: {:.2f}'.format(
        recall_score(y_test, y_pred, average='macro')), file=f)
    print(
        'Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')), file=f)

    print('Weighted Precision: {:.2f}'.format(
        precision_score(y_test, y_pred, average='weighted')), file=f)
    print('Weighted Recall: {:.2f}'.format(
        recall_score(y_test, y_pred, average='weighted')), file=f)
    print(
        'Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')), file=f)

    print('\nClassification Report\n', file = f)
    print(classification_report(y_test, y_pred, target_names=classes), file=f)
    f.close()

In [ ]:
fig = plt.figure(figsize=(10, 6))
plot_graph(list(range(0, NUM_ROUNDS)), tff_train_acc, label='Train Accuracy')
plot_graph(list(range(0, NUM_ROUNDS)), tff_val_acc,
           label='Validation Accuracy')
plt.legend()
plt.savefig(output_dir / "federated_model_Accuracy.png")

plt.figure(figsize=(10, 6))
plot_graph(list(range(0, NUM_ROUNDS)), tff_train_loss, label='Train loss')
plot_graph(list(range(0, NUM_ROUNDS)), tff_val_loss, label='Validation loss')
plt.legend()
plt.savefig(output_dir / "federated_model_loss.png")


## Single Model with all Data at once (for comparison)

### Data Loading

In [ ]:
# train_data = train_data[0].concatenate(train_data[1])
# val_data = val_data[0].concatenate(val_data[1])


### Model Definition

In [ ]:
# def model_fn():
#     model = tf.keras.models.Sequential([
#         tf.keras.layers.InputLayer(input_shape=(4,)),
#         tf.keras.layers.Dense(32, activation='relu'),
#         tf.keras.layers.Dense(64, activation='relu'),
#         tf.keras.layers.Dense(32, activation='relu'),
#         tf.keras.layers.Dense(3, activation='sigmoid'),
#     ])

#     model.compile(
#         loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#         optimizer=tf.keras.optimizers.Adam(),
#         metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
#     )

#     return model


### Training

In [ ]:
# model = model_fn()
# history = model.fit(train_data, epochs=NUM_ROUNDS,verbose=0)


### Evaluation

In [ ]:
# test_scores = model.evaluate(val_data)
# single_metrics = {
#     'loss': test_scores[0],
#     'accuracy': test_scores[1],
# }
# single_metrics


## Conclusion

Comparing both models:

In [ ]:
# print(f"---Single model metrics---\n{single_metrics}\n")
# print(f"---Federated model metrics---\n{dict(federated_metrics)}")


The Federated Learning approach has a better balance between precision and recall, which might be an indicator of better handling of the imbalanced dataset.